In [1]:
import numpy as np
import tensorflow as tf

In [2]:
data = tf.contrib.learn.datasets.mnist.load_mnist('./')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Parameters
num_steps = 2000 # Total steps to train
batch_size = 1024 # The number of samples per batch
num_classes = 10 # The 10 digits
num_features = 784 # Each image is 28x28 pixels
num_trees = 10
max_nodes = 1000

# Input and Target data
X = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
Y = tf.placeholder(tf.int32, shape=[None])

In [4]:
from tensorflow.contrib.tensor_forest.python import tensor_forest

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)

# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)

correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_output_columns': 11, 'feature_bagging_fraction': 1.0, 'valid_leaf_threshold': 1, 'checkpoint_stats': False, 'initialize_average_splits': False, 'pruning_type': 0, 'prune_every_samples': 0, 'dominate_fraction': 0.99, 'max_fertile_nodes': 0, 'early_finish_check_every_samples': 0, 'dominate_method': 'bootstrap', 'bagging_fraction': 1.0, 'regression': False, 'param_file': None, 'bagged_num_features': 784, 'use_running_stats_method': False, 'max_nodes': 1000, 'split_finish_name': 'basic', 'leaf_model_type': 0, 'stats_model_type': 0, 'bagged_features': None, 'num_features': 784, 'split_after_samples': 250, 'num_outputs': 1, 'collate_examples': False, 'split_type': 0, 'num_classes': 10, 'num_splits_to_consider': 28.0, 'split_name': 'less_or_equal', 'finish_type': 0, 'inference_tree_paths': False, 'split_pruning_name': 'none', 'base_random_seed': 0, 'num_trees': 10, 'model_name': 'all_dense'}
Instructions for updating:
Co

In [5]:
from tensorflow.python.ops import resources

# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
    resources.initialize_resources(resources.shared_resources()))

# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars)

# Training
for i in range(1, num_steps + 1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x, batch_y = data.train.next_batch(batch_size)
    _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

# Test Model
test_x, test_y = data.test.images, data.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Step 1, Loss: -1.000000, Acc: 0.332031
Step 50, Loss: -257.399994, Acc: 0.866211
Step 100, Loss: -539.599976, Acc: 0.913086
Step 150, Loss: -831.599976, Acc: 0.926758
Step 200, Loss: -1001.000000, Acc: 0.916992
Step 250, Loss: -1001.000000, Acc: 0.919922
Step 300, Loss: -1001.000000, Acc: 0.924805
Step 350, Loss: -1001.000000, Acc: 0.931641
Step 400, Loss: -1001.000000, Acc: 0.913086
Step 450, Loss: -1001.000000, Acc: 0.929688
Step 500, Loss: -1001.000000, Acc: 0.927734
Step 550, Loss: -1001.000000, Acc: 0.936523
Step 600, Loss: -1001.000000, Acc: 0.928711
Step 650, Loss: -1001.000000, Acc: 0.936523
Step 700, Loss: -1001.000000, Acc: 0.937500
Step 750, Loss: -1001.000000, Acc: 0.908203
Step 800, Loss: -1001.000000, Acc: 0.919922
Step 850, Loss: -1001.000000, Acc: 0.903320
Step 900, Loss: -1001.000000, Acc: 0.940430
Step 950, Loss: -1001.000000, Acc: 0.925781
Step 1000, Loss: -1001.000000, Acc: 0.912109
Step 1050, Loss: -1001.000000, Acc: 0.911133
Step 1100, Loss: -1001.000000, Acc: 0.9